In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import json

# Cargar los archivos JSON que contienen los conjuntos de entrenamiento y prueba
with open('/content/ner_promed_train.json') as f:
    train_data = json.load(f)

with open('/content/ner_promed_test.json') as f:
    test_data = json.load(f)

# Preparar los datos
# Aquí deberás realizar el preprocesamiento necesario para convertir las etiquetas a índices numéricos,
# dividir los datos en oraciones y etiquetas, y crear el diccionario de vocabulario.

# Ejemplo de preprocesamiento para convertir etiquetas a índices numéricos
label_to_index = {
    'O': 0,
    'B-Abreviatura': 1,
    'I-Abreviatura': 2,
    'B-Conditional': 3,
    'I-Conditional': 4,
    'B-Date': 5,
    'I-Date': 6,
    'B-Disease': 7,
    'I-Disease': 8,
    'B-Host': 9,
    'I-Host': 10,
    'B-Location': 11,
    'I-Location': 12,
    'B-Negation': 13,
    'I-Negation': 14,
    'B-Number_of_cases': 15,
    'I-Number_of_cases': 16,
    'B-Origin': 17,
    'I-Origin': 18,
    'B-Past': 19,
    'I-Past': 20,
    'B-Transmission_form': 21,
    'I-Transmission_form': 22,
    'B-Uncertainty': 23,
    'I-Uncertainty': 24,
    'B-negatesRel': 25,
    'I-negatesRel': 26,
    'B-speculatesNrCasesOfDis': 27,
    'I-speculatesNrCasesOfDis': 28,
    'B-speculatesRel': 29,
    'I-speculatesRel': 30,
    'B-speculatesTFDs': 31,
    'I-speculatesTFDs': 32
}

# Obtener oraciones y etiquetas de los datos de entrenamiento
train_sentences = [data["sentence"] for data in train_data]
train_labels = [data["labels"] for data in train_data]

# Convertir etiquetas a índices numéricos en los datos de entrenamiento
train_labels_indices = [[label_to_index[label] for label in labels] for labels in train_labels]

# Crear el diccionario de vocabulario
vocab = set([token for sentence in train_sentences for token in sentence])
vocab_size = len(vocab)
word_to_index = {word: index for index, word in enumerate(vocab)}

# Definir parámetros del modelo
embedding_dim = 100
hidden_dim = 128
output_dim = len(label_to_index)



In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import json

# Cargar los archivos JSON que contienen los conjuntos de entrenamiento y prueba
train_file = '/content/ner_promed_train.json'
test_file = '/content/ner_promed_test.json'

with open(train_file) as f:
    train_data = json.load(f)

with open(test_file) as f:
    test_data = json.load(f)

# Preparar los datos
label_to_index = {
    'O': 0,
    'B-Abreviatura': 1,
    'I-Abreviatura': 2,
    'B-Conditional': 3,
    'I-Conditional': 4,
    'B-Date': 5,
    'I-Date': 6,
    'B-Disease': 7,
    'I-Disease': 8,
    'B-Host': 9,
    'I-Host': 10,
    'B-Location': 11,
    'I-Location': 12,
    'B-Negation': 13,
    'I-Negation': 14,
    'B-Number_of_cases': 15,
    'I-Number_of_cases': 16,
    'B-Origin': 17,
    'I-Origin': 18,
    'B-Past': 19,
    'I-Past': 20,
    'B-Transmission_form': 21,
    'I-Transmission_form': 22,
    'B-Uncertainty': 23,
    'I-Uncertainty': 24,
    'B-negatesRel': 25,
    'I-negatesRel': 26,
    'B-speculatesNrCasesOfDis': 27,
    'I-speculatesNrCasesOfDis': 28,
    'B-speculatesRel': 29,
    'I-speculatesRel': 30,
    'B-speculatesTFDs': 31,
    'I-speculatesTFDs': 32
}

def prepare_data(data):
    sentences = []
    labels = []
    for item in data["samples"]:
        tokens = item["tokens"]
        tags = item["tags"]
        sentences.append(tokens)
        label_indices = [label_to_index[tag] for tag in tags]
        labels.append(label_indices)
    return sentences, labels


train_sentences, train_labels = prepare_data(train_data)
test_sentences, test_labels = prepare_data(test_data)


# Crear el diccionario de vocabulario
vocab = set([token for sentence in train_sentences for token in sentence])
vocab_size = len(vocab)
word_to_index = {word: index for index, word in enumerate(vocab)}

# Función para convertir oraciones a secuencias de índices numéricos
def sentence_to_indices(sentence):
    return [word_to_index.get(word, 0) for word in sentence]



#
#El error ValueError: expected sequence of length 679 at dim 1 (got 281)
#indica que las secuencias de entrada tienen longitudes diferentes.
#Esto puede ocurrir si las oraciones en el conjunto de prueba tienen
#longitudes distintas a las oraciones en el conjunto de entrenamiento.
#
#Para solucionar este problema, puedes aplicar el método de padding a
#las secuencias de entrada para que todas tengan la misma longitud.
#Esto se puede lograr utilizando la función torch.nn.utils.rnn.pad_sequence.
#


from torch.nn.utils.rnn import pad_sequence

# Convertir las oraciones a secuencias de índices numéricos
train_sentences_indices = [torch.tensor(sentence_to_indices(sentence), dtype=torch.long) for sentence in train_sentences]
test_sentences_indices = [torch.tensor(sentence_to_indices(sentence), dtype=torch.long) for sentence in test_sentences]

# Aplicar padding a las secuencias de entrada
train_sentences_padded = pad_sequence(train_sentences_indices, batch_first=True)
test_sentences_padded = pad_sequence(test_sentences_indices, batch_first=True)

# Convertir las secuencias de entrada a tensores
train_sentences_padded = torch.stack(train_sentences_padded)
test_sentences_padded = torch.stack(test_sentences_padded)


#En este código, pad_sequence toma una lista de tensores de secuencias
# y aplica padding para que todas tengan la misma longitud en la
# dimensión especificada (en este caso, la dimensión del lote,
#que se establece en batch_first=True).

# Definir parámetros del modelo
embedding_dim = 100
hidden_dim = 128
output_dim = len(label_to_index)

# Definir la arquitectura del modelo RNN en PyTorch
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        output = self.fc(output)
        return output

# Instanciar el modelo
model = RNNModel(vocab_size, embedding_dim, hidden_dim, output_dim)

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
num_epochs = 10
batch_size = 32

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for i in range(0, len(train_sentences_indices), batch_size):
        batch_sentences = train_sentences_indices[i:i+batch_size]
        batch_labels = train_labels[i:i+batch_size]

        optimizer.zero_grad()

        inputs = torch.tensor(batch_sentences, dtype=torch.long)
        labels = torch.tensor(batch_labels, dtype=torch.long)

        outputs = model(inputs)
        outputs = outputs.view(-1, output_dim)
        labels = labels.view(-1)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}')

# Evaluación del modelo
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for i in range(0, len(test_sentences_indices), batch_size):
        batch_sentences = test_sentences_indices[i:i+batch_size]
        batch_labels = test_labels[i:i+batch_size]

        inputs = torch.tensor(batch_sentences, dtype=torch.long)
        labels = torch.tensor(batch_labels, dtype=torch.long)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, dim=-1)

        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print(f'Test Accuracy: {accuracy * 100:.2f}%')


TypeError: ignored